# Étape 4 – Optimisation des hyperparamètres et du seuil métier

Dans ce notebook, nous allons :

- Optimiser les hyperparamètres d’un modèle avec **Optuna**,
- Définir une fonction de coût métier (coût des FN > coût des FP),
- Ajuster le **seuil de décision** pour minimiser ce coût métier.

**Objectif :** Un modèle robuste, adapté aux enjeux métier, traçable avec MLflow.

### 1. Chargement des bibliothèques et des données

Dans cette première cellule, nous importons les bibliothèques nécessaires (pandas, numpy, Optuna, XGBoost, MLflow, etc.)  
Nous chargeons également les jeux de données `X_train`, `X_test`, `y_train`, `y_test` à partir de fichiers `.parquet`.

In [ ]:
import pandas as pd
import numpy as np
import optuna
import mlflow
import mlflow.sklearn
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

X_train = pd.read_parquet("../data/output/X_train.parquet")
X_test = pd.read_parquet("../data/output/X_test.parquet")
y_train = pd.read_parquet("../data/output/y_train.parquet").squeeze()
y_test = pd.read_parquet("../data/output/y_test.parquet").squeeze()

### 2. Définition de la fonction de coût métier

Dans cette section, nous définissons une fonction qui calcule le **coût total métier** en pondérant différemment les faux négatifs (FN) et les faux positifs (FP). Ici, nous considérons qu’un FN coûte 10 fois plus cher qu’un FP.

In [ ]:
def compute_business_cost(y_true, y_pred, cost_fn=10, cost_fp=1):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    return fn * cost_fn + fp * cost_fp

### 3. Définition de l’objectif pour Optuna

Nous construisons une fonction `objective(trial)` qui teste plusieurs combinaisons d’hyperparamètres pour XGBoost.  
À chaque itération, le modèle est évalué selon le coût métier à un seuil de 0.5 (à optimiser plus tard).

In [ ]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "max_depth": trial.suggest_int("max_depth", 2, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "eval_metric": "logloss"
    }

    model = XGBClassifier(**params)
    model.fit(X_train, y_train)
    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_proba >= 0.5).astype(int)

    return compute_business_cost(y_test, y_pred)

### 4. Lancement de l’optimisation avec Optuna

Nous lançons l’étude Optuna pour minimiser notre fonction de coût métier.  
Les meilleurs hyperparamètres seront utilisés pour entraîner un modèle final.

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=9)

print("Best params:", study.best_params)
print("Best cost:", study.best_value)

### 5. Optimisation du seuil métier

Une fois les meilleurs hyperparamètres trouvés, nous testons plusieurs seuils de décision (de 0.1 à 0.9)  
afin de minimiser le coût métier et choisir le seuil optimal pour notre classification.

In [ ]:
model = XGBClassifier(**study.best_params, eval_metric="logloss")
model.fit(X_train, y_train)
y_proba = model.predict_proba(X_test)[:, 1]

thresholds = np.arange(0.1, 0.9, 0.05)
costs = []

for t in thresholds:
    y_pred = (y_proba >= t).astype(int)
    cost = compute_business_cost(y_test, y_pred)
    costs.append(cost)

optimal_threshold = thresholds[np.argmin(costs)]
print("Seuil optimal :", optimal_threshold)

### 6. Enregistrement du modèle et du seuil avec MLflow

Nous traçons l’expérimentation avec MLflow, incluant :
- Les hyperparamètres,
- Le seuil optimal,
- Le coût final,
- Et le modèle entraîné.

Cela permet de garder une trace pour de futures industrialisations.

In [ ]:
import mlflow.models

mlflow.set_tracking_uri("file:../mlruns")
mlflow.set_experiment("modele_optimisation")

# Exemple de signature et input
from mlflow.models.signature import infer_signature
input_example = X_test.iloc[:5]
signature = infer_signature(X_test, model.predict_proba(X_test))

with mlflow.start_run():
    mlflow.log_params(study.best_params)
    mlflow.log_metric("optimal_threshold", optimal_threshold)
    mlflow.log_metric("best_cost", min(costs))
    mlflow.sklearn.log_model(model, "model", signature=signature, input_example=input_example)
    mlflow.set_tag("model_type", "XGBoost optimisé avec Optuna")

### 7. Tester plusieurs seuils et tracer la courbe coût métier vs seuil

Nous évaluons les performances du modèle pour plusieurs seuils de décision entre 0.05 et 0.95. À chaque seuil, nous transformons les probabilités en prédictions binaires et calculons le coût métier associé.

In [ ]:
import matplotlib.pyplot as plt

thresholds = np.arange(0.05, 0.96, 0.01)
costs = []

y_proba = model.predict_proba(X_test)[:, 1]

for threshold in thresholds:
    y_pred_thresh = (y_proba >= threshold).astype(int)
    cost = compute_business_cost(y_test, y_pred_thresh, cost_fn=10, cost_fp=1)
    costs.append(cost)

best_threshold = thresholds[np.argmin(costs)]
min_cost = min(costs)

plt.figure(figsize=(10, 5))
plt.plot(thresholds, costs, marker='o')
plt.axvline(best_threshold, color='red', linestyle='--', label=f'Seuil optimal = {best_threshold:.2f}')
plt.title("Coût métier en fonction du seuil de décision")
plt.xlabel("Seuil de classification")
plt.ylabel("Coût métier (FN*10 + FP*1)")
plt.legend()
plt.grid(True)
plt.show()

print(f"Seuil optimal métier : {best_threshold:.2f} avec un coût total de {min_cost:.0f}")

### Conclusion

Dans ce notebook, nous avons optimisé notre modèle en intégrant pleinement les enjeux métier, en particulier le coût différencié des erreurs (FN > FP). Nous avons :
- Utilisé Optuna pour rechercher automatiquement les meilleurs hyperparamètres d’un modèle XGBoost,
- Défini une fonction de coût personnalisée pondérant différemment les faux positifs et faux négatifs,
- Testé plusieurs seuils de classification entre 0.1 et 0.9,
- Tracé la courbe du coût métier en fonction du seuil,
- Identifié un seuil optimal à {best_threshold:.2f}, minimisant le coût total des erreurs à {min_cost:.0f},
- Enregistré les expérimentations avec MLflow pour assurer leur traçabilité, comparabilité et réutilisabilité.

Ce travail permet de sélectionner un modèle robuste et contextualisé, mieux aligné avec les objectifs métier. En remplaçant le seuil par défaut (0.5) par un seuil optimisé, nous posons une base solide pour les prochaines étapes : industrialisation, suivi en production et décision fiable en conditions réelles.


## Points de vigilance

- ⚠️ Ne pas garder le seuil 0.5 sans justification : il ne reflète pas les enjeux métier.
- 📉 Tracer le coût métier vs seuil est indispensable pour bien choisir.
- 🎯 Ne pas se contenter de l’AUC ou accuracy.
- 📊 Toujours adapter les métriques et objectifs au contexte métier.
- 🔁 Penser à tester la robustesse du modèle sur d’autres splits / seeds.